In [1]:
import pandas as pd 
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
from matplotlib import pyplot as plt
import re
from nltk.corpus import stopwords


In [2]:
test = pd.read_csv("nlp_test.csv")
train = pd.read_csv("nlp_train.csv")

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5346 entries, 0 to 5345
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        5346 non-null   int64 
 1   keyword   5311 non-null   object
 2   location  3590 non-null   object
 3   text      5346 non-null   object
 4   target    5346 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 209.0+ KB


In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2267 entries, 0 to 2266
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        2267 non-null   int64 
 1   keyword   2241 non-null   object
 2   location  1490 non-null   object
 3   text      2267 non-null   object
dtypes: int64(1), object(3)
memory usage: 71.0+ KB


In [5]:
train_2 = train.drop(["id","location"],axis=1)
test_2 = test.drop(["location"],axis=1)
train_2.head(20)

,keyword,text,target
0,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,NaN,Forest fire near La Ronge Sask. Canada,1
2,NaN,All residents asked to 'shelter in place' are ...,1
3,NaN,#RockyFire Update => California Hwy. 20 closed...,1
4,NaN,I'm on top of the hill and I can see a fire in...,1
5,NaN,Three people died from the heat wave so far,1
6,NaN,#Flood in Bago Myanmar #We arrived Bago,1
7,NaN,Damage to school bus on 80 in multi car crash ...,1
8,NaN,What's up man?,0
9,NaN,I love fruits,0


In [6]:
test_2.head(20)

,id,keyword,text
0,6,NaN,"13,000 people receive #wildfires evacuation or..."
1,7,NaN,Just got sent this photo from Ruby #Alaska as ...
2,10,NaN,#flood #disaster Heavy rain causes flash flood...
3,14,NaN,There's an emergency evacuation happening now ...
4,15,NaN,I'm afraid that the tornado is coming to our a...
5,17,NaN,Haha South Tampa is getting flooded hah- WAIT ...
6,18,NaN,#raining #flooding #Florida #TampaBay #Tampa 1...
7,28,NaN,What a goooooooaaaaaal!!!!!!
8,36,NaN,LOOOOOOL
9,37,NaN,No way...I can't eat that shit


In [7]:
train_2 = train_2.fillna('')
test_2 = test_2.fillna('')
train_2.head(20)

,keyword,text,target
0,,Our Deeds are the Reason of this #earthquake M...,1
1,,Forest fire near La Ronge Sask. Canada,1
2,,All residents asked to 'shelter in place' are ...,1
3,,#RockyFire Update => California Hwy. 20 closed...,1
4,,I'm on top of the hill and I can see a fire in...,1
5,,Three people died from the heat wave so far,1
6,,#Flood in Bago Myanmar #We arrived Bago,1
7,,Damage to school bus on 80 in multi car crash ...,1
8,,What's up man?,0
9,,I love fruits,0


In [8]:
test_2.head(20)

,id,keyword,text
0,6,,"13,000 people receive #wildfires evacuation or..."
1,7,,Just got sent this photo from Ruby #Alaska as ...
2,10,,#flood #disaster Heavy rain causes flash flood...
3,14,,There's an emergency evacuation happening now ...
4,15,,I'm afraid that the tornado is coming to our a...
5,17,,Haha South Tampa is getting flooded hah- WAIT ...
6,18,,#raining #flooding #Florida #TampaBay #Tampa 1...
7,28,,What a goooooooaaaaaal!!!!!!
8,36,,LOOOOOOL
9,37,,No way...I can't eat that shit


In [9]:
test_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2267 entries, 0 to 2266
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       2267 non-null   int64 
 1   keyword  2267 non-null   object
 2   text     2267 non-null   object
dtypes: int64(1), object(2)
memory usage: 53.3+ KB


In [10]:
train_2['text']  = train_2['text']+ train_2['keyword']
test_2['text']  = test_2['text']+ test_2['keyword']
test_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2267 entries, 0 to 2266
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       2267 non-null   int64 
 1   keyword  2267 non-null   object
 2   text     2267 non-null   object
dtypes: int64(1), object(2)
memory usage: 53.3+ KB


In [11]:
signos = re.compile("(\.)|(\;)|(\:)|(\!)|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)|(\=)|(\>)|(\<)|(#)|(you)|(me)|(yours)|(your)|(our)|(she)|(he)|(us)|(haha)|(i'm)|(what's)|(rt)|('re)|('v)")

def signs_tweets(tweet):
    return signos.sub('', tweet.lower())

train_2['text'] = train_2['text'].apply(signs_tweets)
test_2['text'] = test_2['text'].apply(signs_tweets)

def remove_links(df):
    return " ".join(['{link}' if ('http') in word else word for word in df.split()])

train_2['text'] = train_2['text'].apply(remove_links)
test_2['text'] = test_2['text'].apply(remove_links)
train_2['text'].head(20)
test_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2267 entries, 0 to 2266
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       2267 non-null   int64 
 1   keyword  2267 non-null   object
 2   text     2267 non-null   object
dtypes: int64(1), object(2)
memory usage: 53.3+ KB


In [12]:
spanish_stopwords = stopwords.words("english")

def remove_stopwords(train_2):
    return " ".join([word for word in train_2.split() if word not in spanish_stopwords])

train_2["text"] = train_2["text"].apply(remove_stopwords)
test_2["text"] = test_2["text"].apply(remove_stopwords)
train_2.head(20)
test_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2267 entries, 0 to 2266
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       2267 non-null   int64 
 1   keyword  2267 non-null   object
 2   text     2267 non-null   object
dtypes: int64(1), object(2)
memory usage: 53.3+ KB


In [13]:
train_2.head(23)

,keyword,text,target
0,,deeds reason eahquake may allah forgive,1
1,,forest fire near la ronge sask canada,1
2,,residents asked 'lter place' notified officers...,1
3,,rockyfire update california hwy closed directi...,1
4,,top hill see fire woods,1
5,,three people died wave far,1
6,,flood bago myanmar arrived bago,1
7,,damage school b multi car crash breaking,1
8,,man,0
9,,love fruits,0


In [14]:
def clean_text(X):
    a = X.split()
    X_new = [x for x in a if not x.startswith("@")]
    X_new = [x for x in a if not x.startswith("{link}")]
    frase = ' '.join(X_new)
    return frase

train_2['text'] = train_2['text'].apply(clean_text)
test_2['text'] = test_2['text'].apply(clean_text)

In [15]:
train_2.head(30)
test_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2267 entries, 0 to 2266
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       2267 non-null   int64 
 1   keyword  2267 non-null   object
 2   text     2267 non-null   object
dtypes: int64(1), object(2)
memory usage: 53.3+ KB


In [16]:
train_2 = train_2.drop(['keyword'],axis=1)
test_2 = test_2.drop(['keyword'],axis=1)

In [17]:
test_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2267 entries, 0 to 2266
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2267 non-null   int64 
 1   text    2267 non-null   object
dtypes: int64(1), object(1)
memory usage: 35.5+ KB


In [18]:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import xml.etree.ElementTree as ET
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [169]:
vectorizer = CountVectorizer()
scv = SVC()

pipeline = Pipeline([('vect', vectorizer), ('svc', SVC())])


# Aqui definimos el espacio de parámetros a explorar
parameters = {
    'vect__max_df': ( 1.0, 0.8),
    'vect__min_df': (20,15, 18),
    'vect__max_features': (400,500,600),
    'vect__ngram_range': ((1,1),(1,2)),  # unigramas or bigramas
    'svc__kernel': ('linear', 'rbf', 'sigmoid'),
    'svc__C': [0.001, 0.1, 0.5, 1, 5, 10],
    'svc__gamma': ('scale', 'auto')
}

grid_search = GridSearchCV(pipeline,
                           parameters,
                           cv=10,
                           n_jobs=1,
                           scoring= 'roc_auc')

In [170]:
grid_search.fit(train_2['text'], train_2['target'])

/Users/cmg/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.71676056 0.71499491 0.71579554 0.71512737 0.71800739 0.71333467
 0.71676056 0.71627429 0.71136373 0.71188748 0.71968582 0.71929411
 0.71676056 0.71627429 0.71252457 0.71277796 0.71968582 0.71929411
 0.71676056 0.71499491 0.71579554 0.71512737 0.71800739 0.71333467
 0.71676056 0.71627429 0.71136373 0.71188748 0.71968582 0.71929411
 0.71676056 0.71627429 0.71252457 0.71277796 0.71968582 0.71929411
        nan        nan        nan        na

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('reglog', LogisticRegression())]),
             n_jobs=-1,
             param_grid={'reglog__C': array([1.00000000e+00, 2.78255940e+00, 7.74263683e+00, 2.15443469e+01,
       5.99484250e+01, 1.66810054e+02, 4.64158883e+02, 1.29154967e+03,
       3.59381366e+03, 1.00000000e+04]),
                         'reglog__penalty': ['l1', 'l2'],
                         'vect__max_df': (1.0, 0.8),
                         'vect__max_features': (400, 500, 600),
                         'vect__min_df': (20, 15, 18),
                         'vect__ngram_range': ((1, 1), (1, 2))},
             scoring='roc_auc')

In [171]:
grid_search.best_estimator_

Pipeline(steps=[('vect', CountVectorizer(max_features=500, min_df=18)),
                ('reglog', LogisticRegression())])

In [56]:
best_grids = [(i, j.best_score_) for i, j in grids.items()]
best_grids = pd.DataFrame(best_grids,
                          columns=['Grid',
                                   'Best score']).sort_values(by='Best score',
                                                              ascending=False)

NameError: name 'grids' is not defined

In [172]:
grid_search.best_score_

0.7196858193133558

In [173]:
predictions_proba = grid_search.predict_proba(test_2['text'])

In [174]:
test_2.count()

id      2267
text    2267
dtype: int64

In [175]:
submission = test_2[['id']]
submission['target']= predictions_proba[:,1]

submission.to_csv('sumission.csv', index =False)